# Introduction to Partial Differential Equations
---

## Chapter 1: Preliminaries (Calculus, Linear Algebra, ODEs, and Python)
---

## Creative Commons License Information
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/80x15.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Introduction to Partial Differential Equations: Theory and Computations</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" property="cc:attributionName" rel="cc:attributionURL">Troy Butler</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" rel="dct:source">https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations</a>.

## Section 1.1: Calculus, Symbolic Computations, and Manufactured Solutions
---

This course makes extensive use of prerequisite knowledge from

- [Multivariable Calculus](#calculus)
  
> This is what puts the "partial" in partial differential equations (PDEs). This is the focus of the current notebook.

- (Basics of) Ordinary Differential Equations (ODEs)

> This is fundamental to understanding numerical time-stepping procedures for spatial-temporal PDEs. We review important aspects of this in the [Chp1Sec3](Chp1Sec3.ipynb) and [Chp1Sec4](Chp1Sec4.ipynb) notebooks.

3. (Basics of) Linear Algebra 

> We provide review in the [Chp1Sec5](Chp1Sec5.ipynb) notebook, which gives students more useful information related to Python and specifically using `numpy` and its linear algebra subpackage. Linear algebra provides many useful perspectives in terms of linear operators acting on vector spaces that motivate many conversations we will have throughout this course. 


---
### <a id="calculus">Section 1.1.1: Summary of important differential calculus concepts</a>
---

- [Partial Derivatives (first, higher order)](https://en.wikipedia.org/wiki/Partial_derivative) are at the heart of PDEs. 

  These allow us to define and understand what a partial differential equation is and to verify a given function is in fact a solution by "plugging it into" the equation. This is particularly crucial when we verify code is performing as expected through the [**method of manufactured solutions**](#manufactured) solutions. 

Some notation review: 

- If $u:\mathbb{R}^3\to\mathbb{R}$, we often write $u(x,y,z)$ and its first-order partial derivatives are often denoted by $\partial_x u, \partial_y u, $ and $\partial_z u$ or more compactly as $u_x$, $u_y$, $u_z$. Other notations are also common (e.g., see https://en.wikipedia.org/wiki/Partial_derivative).

- The [del or nabla](https://en.wikipedia.org/wiki/Del) operator, $\nabla$, is a vector differential operator applied to scalar-valued or vector-valued functions: $\nabla u$ (defining the gradient), $\nabla \times \vec{u}$ (defining the curl), and $\nabla \cdot \vec{u}$ (defining the divergence) where $u$ is a scalar-valued function and $\vec{u}$ is a vector-valued function. We do not use the curl in this class, so we summarize only the gradient and divergence below.

  - On $\mathbb{R}^n$, the differential operator $\nabla$ and [gradient](https://en.wikipedia.org/wiki/Gradient) of $u:\mathbb{R}^n\to\mathbb{R}$ are defined as
<br><br>
$$
    \nabla := \begin{bmatrix}
                \partial_{x_1} \\
                \partial_{x_2} \\
                \vdots \\
                \partial_{x_n}
                \end{bmatrix}, \ \text{ and } \ 
      \nabla u := \left(\begin{array}{c}
                \partial_{x_1}u \\
                \partial_{x_2}u \\
                \vdots \\
                \partial_{x_n}u
                \end{array}\right), \ \text{respectively.}
$$
<br><br>
  - For $\vec{u}:\mathbb{R}^n\to\mathbb{R}^n$, its [divergence](https://en.wikipedia.org/wiki/Divergence) is given by
<br><br>
$$
    \nabla \cdot \vec{u} :=  \sum_{i=1}^n \partial_{x_i}\vec{u}_i
$$
<br><br>
    where  $\vec{u}_i$ denotes the $i$th component of the vector $\vec{u}$.
<br><br>

- [Taylor's theorem](https://en.wikipedia.org/wiki/Taylor%27s_theorem) allows us to understand why finite difference schemes are valid numerical schemes for approximating solutions.

---
### Section 1.1.2: Symbolic differentiation in Python
---

We make use of the [`sympy`](https://www.sympy.org/en/index.html) package in Python to symbolically differentiate/integrate and manipulate functions via simplification or factoring routines. 

This allows us to (1) check any of our by-hand work, and (2) get results quicker!

Below, we focus just on the differentiation aspect.

In [ ]:
import sympy as sp  # First we need to import sympy, we use the standard sp abbreviation

Below we create symbolic variables `x` and `y`.

In [ ]:
x, y = sp.symbols('x, y')  # Creating x and y as symbols

Now we use symbolic functions available within `sympy` to create the symbolic function for $$u=e^{-x^2}\sin(y) + x^2 - y + xy.$$

In [ ]:
u = sp.exp(-x**2) * sp.sin(y) + x**2 - y + x*y

In [ ]:
u  # The standard output for a symbolic function is "pretty"

We now use sympy to compute the first-order partial derivatives of u with respect to $x$ and $y$.

In [ ]:
u.diff(x,1)  # The x indicates the variable to differentiate with respect to and 1 indicates the order

In [ ]:
u.diff(y,1)

Higher order derivative computations are just as easily computed.

In [ ]:
u.diff(x,5)  # This computes a 5th order derivative. Who wants to do this by hand?

Since a vector is a type of matrix, we can use symbolic matrices to represent gradients.

In [ ]:
u_grad = sp.Matrix([u.diff(x,1), u.diff(y,1)])

In [ ]:
u_grad

`sympy` matrices have a Jacobian method attribute that can also be used to produce the gradient as follows. Note the use of the transpose at the end since a Jacobian computes a matrix of first-order derivatives for a vector-valued function where each row is the gradient of the corresponding component in the vector-valued function.

In [ ]:
sp.Matrix([u]).jacobian(sp.Matrix([x, y])).T

#### Some special characters

What about Greek letters used as coefficients or their own variables? Luckily, there is the [`abc`](https://docs.sympy.org/latest/modules/abc.html) module within `sympy`.

In [ ]:
from sympy.abc import pi, rho, alpha, kappa, beta

In [ ]:
v = sp.exp(-kappa*x**2) * sp.sin(pi*y) + rho*x**2 - alpha*y + beta*x*y

In [ ]:
v

In [ ]:
sp.Matrix([v]).jacobian(sp.Matrix([x, y])).T 

In [ ]:
sp.Matrix([v]).jacobian(sp.Matrix([x, y, pi, rho, alpha, kappa, beta])).T

---
#### Student activity
---

Consider the following two functions

$$
    f(x,y,z) = xy^2z^3 \ \text{ and } \ g(x,y,z) =  \sin(x)\cos(y)\tan(z) - 3^x.
$$

Use `sympy` to create symbolic functions for the above two functions and compute their first-order partial derivatives as well as their gradients. Several blank code cells are below to get you started, but students should feel free to make more as needed as well as adding Markdown cells for notes.

*Hint: First you will need to create a new symbolic variable z.* 

---
### Section 1.1.3: The Laplacian
---

The **Laplace operator** (often referred to simply as the **Laplacian**) is denoted by $\Delta$ and defined on $\mathbb{R}^n$ by
$$
\Delta := \nabla^2 := \nabla\cdot \nabla= \sum_{i=1}^n \partial_{x_i}^2.
$$

From this definition, it follows that

$$
    \nabla \cdot \nabla u = \nabla \cdot (\nabla u)= \Delta u.
$$

In other words, the Laplacian of a scalar-valued function $u:\mathbb{R}^n\to\mathbb{R}$ is defined by the divergence of the gradient of $u$.

Let's take the Laplacian of the `u` defined as a symbolic function above.

In [ ]:
u = sp.exp(-x**2) * sp.sin(y) + x**2 - y + x*y  # This is just in case we edited u above. It is not necessary to redefine u everytime we want to use it.

In [ ]:
Del_u = u.diff(x,2) + u.diff(y,2)

In [ ]:
Del_u

In [ ]:
# A slick way (good for high dimensions)

Del_u = 0  # initialize the Del_u function as a 0
for var in [x, y]:  # loop through each variable to differentiate
    Del_u += u.diff(var, 2)  # add the second derivative of u with respect to given variable to Del_u

In [ ]:
Del_u

In [ ]:
v = sp.exp(-kappa*x**2) * sp.sin(pi*y) + rho*x**2 - alpha*y + beta*x*y  # Just in case v was redefined above when exploring. Not necessary otherwise

In [ ]:
Del_v = 0
for var in [x, y, rho, beta, alpha, kappa]:
    Del_v += v.diff(var, 2)

In [ ]:
Del_v

---
### Section 1.1.4: Our first manufactured solution
---

In Chapter 2, we will consider solutions to Poisson's problem defined by $-\Delta u = f$ on a given domain with some specified boundary conditions.

For now, we remove the complication of the boundary conditions so that we focus just on $-\Delta u=f$ being satisfied on $\mathbb{R}^n$ for some specified $n$.

In general, we use numerical methods to solve PDEs. By "solve" we mean produce a numerical estimate to $u$ at some set of points discretizing the domain. We do this because in general we cannot produce closed-form solutions to PDEs and must rely on numerical methods. However, we want to be able to *test* code under conditions where we *know* what the solutions are to make sure that the code is producing reasonable approximations to these known solutions. If this is not the case, then we would certainly *not* trust the code to produce useful estimates of solutions in cases where we have no idea what the solution is!

This leads us to what is commonly referred to as the ***method of manufactured solutions.***

The idea is quite simple. We want to solve a problem of a particular type such as $-\Delta u = f$ numerically. The "data" of the problem (here, the $f$) will change in each particular instance, and the "solution" of the problem (here, the $u$) and its numerical approximation will subsequently change as the data varies from instance to instance. To verify the code is working properly via the method of manufactured solutions, we *start* with a solution. This may strange, but it is quite intuitive. Pick a $u$, any $u$, and as long as it satisfies any constraints of the problem (e.g., boundary conditions), you can then plug it into the differential equation to figure out what $f$ would have produced such a $u$. Now, you know the data $f$ that should be "fed" into the code to check if it produces a numerical approximation to the $u$ that you manufactured. That is it. This is the method.

Sometimes it is easier said than done because the constraints of the problem (e.g., boundary and initial conditions) may seemingly force you to "throw" away the function $u$ you wanted to use. We will see how to deal with this in Chapter 2 by simply "adjusting" the $u$ (usually through the inclusion of a linear function to adjust its boundary values) to create a function that does serve as a manufactured solution.

This illustrates a good use of a symbolic toolbox because we can ensure that all the derivative computations are done correctly (as long as we coded them correctly). 

In [ ]:
f = - Del_u  # The process of defining the data associated with a manufactured solution 

In [ ]:
f

Yes, it was as easy as what was done above. Now, we know exactly what function $f$ to put into any purported solver for $-\Delta u =f$ on $\mathbb{R}^2$. We can then check the output of the purported solver against what we know $u$ to be.

This is also *incredibly useful* when verifying rates of convergence of numerical methods (something we will study in the [Chp1Sec2](Chp1Sec2.ipynb) notebook).

---
### Section 1.1.5: Some more `sympy` fun
---

`sympy` also has an [`integrals` module](https://docs.sympy.org/latest/modules/integrals/integrals.html) for computing indefinite and definite integrals as well as integral transforms (such as Laplace and Fourier transforms). 

We show some examples below.

In [ ]:
# Note that indefinite integrals do not include the constant of integration 

sp.integrate(x**2 + x*y - y, x)  # Determines an antiderivative of the function $x^2+xy-y$ with respect to x

In [ ]:
sp.integrate(x**2 + x*y - y, y)  # Integrates with respect to y

In [ ]:
sp.integrate(x**2 + x*y - y, (x, 0, 1))  # A definite integral, with respect to x, from 0 to 1

In [ ]:
# What about an iterated integral?

sp.integrate(sp.integrate(x**2 + x*y - y, (x, 0, 1)), (y, 2, 3))

In [ ]:
# Evaluating an iterated integral via a for-loop

f = x**2 + x*y - y
integrand = f
for var, limits in zip([x,y], [(0,1), (2,3)]):
    integrand = sp.integrate(integrand, (var, limits[0], limits[1]))  

# The integral is the final integrand
integral = integrand

In [ ]:
integral

---
#### Navigation:

- [Previous](Chp1Sec0.ipynb)

- [Next](Chp1Sec2.ipynb)
---